## Data pipeline for predicting risk of mortality.

## Imports and Globals

In [ ]:
import os
import random
from typing import Optional, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cyclops.feature_handler import FeatureHandler
from cyclops.plotter import set_bars_color, setup_plot, plot_timeline
from cyclops.processor import run_data_pipeline
from cyclops.processors.aggregate import Aggregator
from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    DIAGNOSIS_CODE,
    DISCHARGE_DISPOSITION,
    DISCHARGE_TIMESTAMP,
    ENCOUNTER_ID,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_CATEGORY,
    HOSPITAL_ID,
    LENGTH_OF_STAY_IN_ER,
    RESTRICT_TIMESTAMP,
    SEX,
    TRIAGE_LEVEL,
    WINDOW_START_TIMESTAMP,
    TIMESTEP,
)
from cyclops.processors.constants import SMH
from cyclops.processors.events import (
    combine_events,
    convert_to_events,
    normalize_events,
)
from cyclops.processors.impute import Imputer
from cyclops.processors.statics import compute_statics
from cyclops.processors.util import (
    fill_missing_timesteps,
    gather_columns,
    pivot_aggregated_events_to_features,
    create_indicator_variables
)
from cyclops.query import gemini
from cyclops.utils.file import save_dataframe, load_dataframe


MORTALITY = "mortality"
LOS = "los"
BASE_DATA_PATH = "/mnt/nfs/project/delirium/drift_exp/_extract"

## Run query, save

In [ ]:
er_admin_table = gemini.get_table(gemini.ER_ADMIN)
encounters = gemini.patient_encounters(
    er_admin_table=er_admin_table,
    years=[2018, 2019, 2020],
    died=True,
    died_binarize_col="mortality",
)
encounters_labs = gemini.events(
    patient_encounters_table=encounters.query, event_category="lab"
)
encounters_vitals = gemini.events(
    patient_encounters_table=encounters.query, event_category="vitals"
)
imaging = gemini.imaging(years=[2018, 2019, 2020])
transfusions = gemini.blood_transfusions(years=[2018, 2019, 2020])
interventions = gemini.interventions()

encounters.run()
encounters_labs.run()
encounters_vitals.run()
imaging.run()
transfusions.run()
interventions.run()

print(f"{len(encounters.data)} rows extracted!")
print(f"{len(encounters_labs.data)} rows extracted!")
print(f"{len(encounters_vitals.data)} rows extracted!")
print(f"{len(imaging.data)} rows extracted!")
print(f"{len(transfusions.data)} rows extracted!")
print(f"{len(interventions.data)} rows extracted!")

encounters_imaging = pd.merge(encounters.data, imaging.data, on=ENCOUNTER_ID, how="inner")
encounters_transfusions = pd.merge(encounters.data, transfusions.data, on=ENCOUNTER_ID, how="inner")
encounters_interventions = pd.merge(encounters.data, interventions.data, on=ENCOUNTER_ID, how="inner")

os.makedirs(BASE_DATA_PATH, exist_ok=True)
encounters.save(os.path.join(BASE_DATA_PATH, "admin_er"))
encounters_labs.save(os.path.join(BASE_DATA_PATH, "labs"))
encounters_vitals.save(os.path.join(BASE_DATA_PATH, "vitals"))
encounters_imaging.to_parquet(os.path.join(BASE_DATA_PATH, "imaging.parquet"))
encounters_transfusions.to_parquet(os.path.join(BASE_DATA_PATH, "transfusions.parquet"))
encounters_interventions.to_parquet(os.path.join(BASE_DATA_PATH, "interventions.parquet"))

## Read saved query data

In [ ]:
encounters_data = pd.read_parquet(os.path.join(BASE_DATA_PATH, "admin_er.parquet"))
labs_data = pd.read_parquet(os.path.join(BASE_DATA_PATH, "labs.parquet"))
# vitals_data = pd.read_parquet(os.path.join(BASE_DATA_PATH, "vitals.parquet"))
imaging_data = pd.read_parquet(os.path.join(BASE_DATA_PATH, "imaging.parquet"))
transfusions_data = pd.read_parquet(os.path.join(BASE_DATA_PATH, "transfusions.parquet"))
interventions_data = pd.read_parquet(os.path.join(BASE_DATA_PATH, "interventions.parquet"))

## Map imaging and transfusions such that they can be treated as events

In [ ]:
imaging_data = imaging_data.rename(columns={"imaging_test_description": "event_name", "performed_date_time": "event_timestamp"})
imaging_data[EVENT_CATEGORY] = "imaging"
imaging_data[EVENT_VALUE] = 1

transfusions_data = transfusions_data.rename(columns={"issue_date_time": "event_timestamp"})
transfusions_data[EVENT_NAME] = transfusions_data["rbc_mapped"]
transfusions_data[EVENT_NAME] = transfusions_data[EVENT_NAME].apply(lambda x: "rbc" if x else "non-rbc")
transfusions_data[EVENT_VALUE] = 1
transfusions_data[EVENT_CATEGORY] = "transfusions"

##  Process interventions such that they can be treated as events

In [ ]:
interventions_data[EVENT_NAME] = ""
interventions_data[EVENT_VALUE] = 1
interventions_data[EVENT_CATEGORY] = "interventions"

for name in ["endoscopy_mapped", "gi_endoscopy_mapped", "bronch_endoscopy_mapped", "dialysis_mapped", "inv_mech_vent_mapped", "surgery_mapped"]:
    cond = interventions_data[name] == 1
    interventions_data[EVENT_NAME][cond] = name.replace("_mapped", "")
interventions_data[EVENT_NAME] = interventions_data[EVENT_NAME].apply(lambda x: "unmapped_intervention" if x == "" else x)
interventions_data = interventions_data[interventions_data["intervention_episode_start_date"].notna()]

interventions_data = interventions_data.rename(columns={"intervention_episode_start_date": EVENT_TIMESTAMP})
interventions_data[EVENT_TIMESTAMP] = interventions_data[EVENT_TIMESTAMP].astype('datetime64[ns]')

## Filter out encounters that had less than 24 hours LOS

In [ ]:
encounters_data[LOS] = encounters_data[DISCHARGE_TIMESTAMP] - encounters_data[ADMIT_TIMESTAMP] + pd.to_timedelta(encounters_data[LENGTH_OF_STAY_IN_ER], unit="h")
encounters_data_atleast_los_24_hrs = encounters_data.loc[encounters_data[LOS] >= pd.to_timedelta(24, unit="h")]
# encounters_data_atleast_los_24_hrs                         

## Get encounters that ended in mortality outcome

In [ ]:
encounters_mortality = encounters_data_atleast_los_24_hrs.loc[encounters_data_atleast_los_24_hrs[MORTALITY] == True]

## Get encounters that didn't end up in mortality outcome (limit to a subset)

In [ ]:
encounters_not_mortality = encounters_data_atleast_los_24_hrs.loc[encounters_data_atleast_los_24_hrs[MORTALITY] == False]

num_encounters_not_mortality = len(encounters_mortality)
encounters_not_mortality_subset = encounters_not_mortality[0:num_encounters_not_mortality]
# encounters_not_mortality_subset

## Combine both subsets

In [ ]:
encounters_train_val_test = pd.concat([encounters_mortality, encounters_not_mortality_subset], ignore_index=True)
# encounters_train_val_test

## Offset discharge timestamp by K hours and create new column

In [ ]:
offset = 0  # in hours
encounters_mortality = encounters_mortality.copy()
encounters_mortality["death_timestamp_offset"] = encounters_mortality[
    DISCHARGE_TIMESTAMP
] + pd.Timedelta(-offset, unit="h")

# encounters_mortality[[DISCHARGE_TIMESTAMP, "death_timestamp_offset"]]

## Convert "death_timestamp_offset" to event

In [ ]:
mortality_risk_events = convert_to_events(
    encounters_mortality, event_name=f"mortality_risk_{offset}", event_category="general", timestamp_col="death_timestamp_offset"
)
mortality_risk_events = pd.merge(
    mortality_risk_events, encounters_mortality, on=ENCOUNTER_ID, how="inner"
)
mortality_risk_events = mortality_risk_events[
    [ENCOUNTER_ID, EVENT_NAME, EVENT_TIMESTAMP, ADMIT_TIMESTAMP, EVENT_VALUE, EVENT_CATEGORY]
]
mortality_risk_events[EVENT_VALUE] = 1
# mortality_risk_events

## Get admission/discharge events

In [ ]:
admit_events = convert_to_events(encounters_train_val_test, event_name="admission", event_category="general", timestamp_col="admit_timestamp")
disch_events = convert_to_events(encounters_train_val_test, event_name="discharge", event_category="general", timestamp_col="discharge_timestamp")

## Filter labs, vitals, imaging, transfusions, interventions to be in train_val_test subset

In [ ]:
labs_train_val_test = labs_data.loc[
    labs_data[ENCOUNTER_ID].isin(encounters_train_val_test[ENCOUNTER_ID])
]
imaging_train_val_test = imaging_data.loc[
    imaging_data[ENCOUNTER_ID].isin(encounters_train_val_test[ENCOUNTER_ID])
]
transfusions_train_val_test = transfusions_data.loc[
    transfusions_data[ENCOUNTER_ID].isin(encounters_train_val_test[ENCOUNTER_ID])
]
interventions_train_val_test = interventions_data.loc[
    interventions_data[ENCOUNTER_ID].isin(encounters_train_val_test[ENCOUNTER_ID])
]

## Normalize all event data

In [ ]:
imaging_events = normalize_events(imaging_train_val_test)
transfusion_events = normalize_events(transfusions_train_val_test)
lab_events = normalize_events(labs_train_val_test)
mortality_risk_events = normalize_events(mortality_risk_events)
intervention_events = normalize_events(interventions_train_val_test)
admit_events = normalize_events(admit_events)
disch_events = normalize_events(disch_events)

## Combine different event data, save

In [ ]:
combined_events = combine_events(
    [
        intervention_events,
        imaging_events,
        transfusion_events,
        lab_events,
        mortality_risk_events,
        admit_events,
        disch_events
    ]
)
save_dataframe(combined_events, os.path.join(BASE_DATA_PATH, "combined_events"))

## Load combined events, aggregate them, save

In [ ]:
combined_events = load_dataframe(os.path.join(BASE_DATA_PATH, "combined_events"))

## Aggregate combined events, save

In [ ]:
aggregator = Aggregator(bucket_size=24, window=144)
aggregated_events = aggregator(combined_events)
save_dataframe(aggregated_events, os.path.join(BASE_DATA_PATH, "aggregated_events"))

## Load aggregated events

In [ ]:
aggregated_events = load_dataframe(os.path.join(BASE_DATA_PATH, "aggregated_events"))

## Pivot table to get column that can be used to create labels. 1 corresponds to timestep where death happens (shifted by offset)

In [ ]:
aggregated_mortality = aggregated_events.loc[aggregated_events[EVENT_NAME] == "death"]
pivoted_mortality = pivot_aggregated_events_to_features(aggregated_mortality, np.mean)
timesteps_mortality = pivoted_mortality.loc[
    pivoted_mortality["death"] == 1
].index.get_level_values(1)

timesteps_mortality_counts = timesteps_mortality.value_counts()
fig, axs = plt.subplots(1, 1, figsize=(10, 5), tight_layout=True)
ts_vals = plt.bar(
    list(np.unique(timesteps_mortality)), list(timesteps_mortality_counts), alpha=0.5
)
set_bars_color(ts_vals, "r")
axs.set_xticks(
    range(len(ts_vals)),
    list(np.unique(timesteps_mortality)),
    rotation="vertical",
    fontsize=20,
)
setup_plot(
    axs,
    "Mortality encounter distribution over timesteps",
    "timestep",
    "Num. encounters that resulted in mortality in that timestep",
    ["Count"],
)
plt.show()

## Pivot aggregated events to get column-wise temporal features and save it

In [ ]:
temporal_features = pivot_aggregated_events_to_features(aggregated_events, np.mean)
save_dataframe(temporal_features, os.path.join(BASE_DATA_PATH, "temporal_features"))

## Add to feature handler, with indicator variables.

In [ ]:
feature_handler = FeatureHandler()

temporal_features = load_dataframe(os.path.join(BASE_DATA_PATH, "temporal_features"))
# indicators = create_indicator_variables(temporal_features)
feature_handler.add_features(temporal_features)
# feature_handler.add_features(indicators)
# feature_handler.drop_features(["death_indicator"])
feature_handler.features["temporal"].columns

In [ ]:
feature_handler.features["temporal"]["unmapped_intervention"].sum()

## Compute static features, save it

In [ ]:
encounters_mortality = gather_columns(
    encounters_mortality,
    [
        ENCOUNTER_ID,
        AGE,
        SEX,
        HOSPITAL_ID,
        ADMIT_TIMESTAMP,
        DISCHARGE_TIMESTAMP,
        TRIAGE_LEVEL,
    ],
)
static_features = compute_statics(encounters_mortality)
save_dataframe(static_features, os.path.join(BASE_DATA_PATH, "static_features"))

##  Load static features, add to feature handler, save all features

In [ ]:
static_features = load_dataframe(os.path.join(BASE_DATA_PATH, "static_features"))
feature_handler.add_features(
    static_features, reference_cols=[HOSPITAL_ID, ADMIT_TIMESTAMP, DISCHARGE_TIMESTAMP]
)
feature_handler.save(BASE_DATA_PATH, "features")

## Create new feature handler, load saved features from file

In [ ]:
feature_handler1 = FeatureHandler()
feature_handler1.load(BASE_DATA_PATH, "features")

## Impute temporal features

In [ ]:
def impute_simple(dataframe: pd.DataFrame, time_index: str=TIMESTEP) -> pd.DataFrame:
    """Impute features using 'Simple' method.
    
    Concatenate the forward filled value, the mask of the measurement,
    and the time of the last measurement.

    Z. Che, S. Purushotham, K. Cho, D. Sontag, and Y. Liu,
    "Recurrent Neural Networks for Multivariate Time Series with Missing Values,"
    Scientific Reports, vol. 8, no. 1, p. 6085, Apr 2018.
    
    Parameters
    ----------
    dataframe: pandas.DataFrame
        Temporal features dataframe.
    time_index: str, optional
        The name of the time-series index.
        
    Returns
    -------
    pandas.DataFrame
     Dataframe after applying imputation.
     
    """
    # Mask missingness.
    masked_df=pd.isna(dataframe)
    masked_df=masked_df.apply(pd.to_numeric)

    # Compute time since last measurement.
    index_of_time=list(dataframe.index.names).index(time_index)
    time_in=[item[index_of_time] for item in dataframe.index.tolist()]
    time_df=dataframe.copy()
    for col in time_df.columns.tolist():
        time_df[col]=time_in
    time_df[masked_df]=np.nan

    # Concatenate the dataframes.
    df_prime=pd.concat([dataframe, masked_df, time_df], axis=1, keys=['measurement','mask', 'time'])
    df_prime.columns=df_prime.columns.rename("impute_simple", level=0)

    # Fill each dataframe using either ffill or mean.
    df_prime=df_prime.fillna(method='ffill').unstack().fillna(0)

    # Swap the levels so that the simple imputation feature is the lowest value.
    col_level_names=list(df_prime.columns.names)
    col_level_names.append(col_level_names.pop(0))

    df_prime=df_prime.reorder_levels(col_level_names, axis=1)
    df_prime.sort_index(axis=1, inplace=True)

    return  df_prime


def create_train_test_split(encounters: pd.DataFrame, fractions: Optional[List] = [0.8, 0.2], split_column: Optional[str] = None, split_values: List = None) -> tuple:
    """Split encounters into train/test.
    
    Parameters
    ----------
    encounters: pandas.DataFrame
        Dataframe with encounter IDs.
    fractions: list, optional
        Fraction of samples to use for train, test sets.
    split_column: str, optional
        If 'split_column' is specified, then that column is used to split.
    split_values: list, optional
        Along with 'split_column', a list of lists can be specified for filtering.
        e.g. [[2008], [2009, 2010]] for train/test split based on year.
    
    Returns
    -------
    tuple
        (train IDs, test IDs)
    
    """
    if split_column:
        if split_column not in encounters.columns:
            raise ValueError("Specified 'split column' not found in input dataframe")
        if not split_values:
            raise ValueError("Specify train/test split values for the 'split column'.!")
        train_encounters = encounters[ENCOUNTER_ID].loc[encounters[split_column].isin(split_values[0])]
        test_encounters = encounters[ENCOUNTER_ID].loc[encounters[split_column].isin(split_values[1])]
        return train_encounters, test_encounters
    
    encounter_ids = list(encounters[ENCOUNTER_ID].unique())
    random.shuffle(encounter_ids)
    num_train = int(fractions[0] * len(encounter_ids))
    
    return encounter_ids[0: num_train], encounter_ids[num_train:]



# By Year
encounters_mortality["year"] = encounters_mortality["admit_timestamp"].dt.year
train_ids, val_test_ids = create_train_test_split(encounters_mortality, split_column="year", split_values=[[2018, 2019], [2020]])

# By HOSPITAL_ID
# train_ids, val_test_ids = create_train_test_split(encounters_mortality, split_column=HOSPITAL_ID, split_values=[["THPM", "SMH", "THPC", "SBK", "MSH"], ["UHNTW", "UHNTG"]])

# Random
# train_ids, val_test_ids = create_train_test_split(encounters_mortality, split_column=HOSPITAL_ID, split_values=[["THPM", "SMH", "THPC"], ["UHNTW", "UHNTG", "MSH"]])

val_ids, test_ids = create_train_test_split(encounters_mortality.loc[encounters_mortality[ENCOUNTER_ID].isin(val_test_ids)], [0.5, 0.5])
print(f"Train set: {len(train_ids)}, Val set: {len(val_ids)}, Test set: {len(test_ids)}")
encounters_mortality[HOSPITAL_ID].value_counts(), encounters_mortality["year"].value_counts()

static = feature_handler1.features["static"]
temporal = feature_handler1.features["temporal"]

mortality_risk_24_targets = temporal["death"].copy()
temporal = temporal.drop(columns="death")

X = temporal[np.in1d(temporal.index.get_level_values(0), static.index.get_level_values(0))]
y = mortality_risk_24_targets[np.in1d(mortality_risk_24_targets.index.get_level_values(0), X.index.get_level_values(0))]

y_train, y_val, y_test = [y[np.in1d(y.index.get_level_values(0), ids)] for ids in [train_ids, val_ids, test_ids]]
X_train, X_val, X_test = [X[np.in1d(X.index.get_level_values(0), ids)] for ids in [train_ids, val_ids, test_ids]]

len(X), len(X_train), len(X_val), len(X_test), len(y), len(y_train), len(y_val), len(y_test)


# X_train_inputs = impute_simple(X_train)
# X_train_inputs = format_dataset(X_train, level="features",imputation_method="simple")
# X_val = X[np.in1d(X.index.get_level_values(0), y_val.index.get_level_values(0))]
# X_val_inputs = format_dataset(X_val, level="features",imputation_method="simple")
# X_test = X[np.in1d(X.index.get_level_values(0), y_test.index.get_level_values(0))]
# X_test_inputs = format_dataset(X_test, level="features",imputation_method="simple")

In [ ]:
encounter_id = random.choice(temporal.index.get_level_values(0))
X_train_inputs.loc[encounter_id]["white blood cell count"]

In [ ]:
encounter_id = random.choice(combined_events["encounter_id"].unique())
# encounter_id = 14235864

# Some strange cases:
# 13759829

# Some interesting cases:
# 13542750
# 15624655
# 15351776
# 15611082
# 15794127

print(encounter_id)
combined_events_encounter = combined_events.loc[combined_events["encounter_id"] == encounter_id]
fig = plot_timeline(combined_events_encounter, return_fig=True)

fig = fig.update_layout(
    width=2000,
    height=800
    
)
fig.show()
# combined_events